Seleccionamos el piso del cual queremos mostrar la información en pantalla, y devolverá un fichero con la informacion requerida.

In [9]:
# Carga de las librerías, funciones, y ficheros necesarios.

import numpy as np 
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.model_selection import train_test_split

# Conecta con google drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset de pisos de airbnb Madrid filtrado (13258 reg) sólo por la ciudad Madrid.
airbnbMadrid = pd.read_csv('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnblistingsMadridv1.csv', sep=';', decimal='.')

# Carga de los dataset de los lugares mas visitados de Madrid.
lugaresInteres = pd.read_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/LugaresInteres.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se introduce por pantalla el numero de piso simulando que es la app de airbnb recopilando la informacion para mostrar al cliente

In [0]:
# Definimos una función para comprobar que el usuario introduce correctamente el ID del piso
def comprueba_int(): 
    while True: 
        try: 
            numEnt = int(input( "Introduzca ID del piso : ")) #Solicita ID piso
            return numEnt 
        except ValueError: 
            print('\nPor favor, introduzca un numero mayor que cero sin decimales\n')
            
# Definimos una función para comprobar que el número introducido sea mayor de 0 : 
def comprueba_mayor_cero(ID): 
    while True: 
        if ID > 0 : 
            return ID
        else:
            print("\nPor favor, introduzca un numero mayor que cero sin decimales\n") 
            ID = comprueba_int()

In [0]:
# Solicita el usuario introduzca el número de identificación del piso. Por ejemplo, el 15332216

IDPiso = 0        # Inicializa el campo a introducir, que deberá ser entero y mayor de cero

#IDPiso = comprueba_mayor_cero(IDPiso)
#print("\nEl ID introducido es :",IDPiso)

IDPiso = 15332216

#airbnbMadridFastFood['ID'][IDPiso]  15332216

In [0]:
import math 

# Función que calcula la distancia geográfica entre dos puntos en metros, pasándole la latitud y la longitud.
def distGeo(latitud1, longitud1, latitud2, longitud2): # posiciones de la vivienda, y los restaurantes
    rad=math.pi/180
    dlatitud=latitud2-latitud1
    dlongitud=longitud2-longitud1
    R=6372.795477598 # Radio de la Tierra
    # La fórmula tiene en cuenta la curvatura del planeta (aunque se podría calcular sin tenerla en cuenta, dado
    # que estamos en la misma ciudad).
    a=(math.sin(rad*dlatitud/2))**2 + math.cos(rad*latitud1)*math.cos(rad*latitud2)*(math.sin(rad*dlongitud/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [0]:
# Lugares de interes
# Para el piso introducido, calculamos la distancia a todos los lugares de interes de Madrid
distanciaMenor = 100  # EL restaurante a menor distancia. Se inicializa a un valor aleatorio. 
latPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Latitude']  # latitud del piso 
lonPiso = airbnbMadrid[airbnbMadrid['ID'] == IDPiso]['Longitude'] # longitud del piso

distancia=abs(distGeo(latPiso,lonPiso,lugaresInteres['Latit'][0],lugaresInteres['Longit'][0]))

# Preparamos el primer dato de los lugares de interes
data=[[lugaresInteres['ID'][0]],[lugaresInteres['Lugar'][0]],[lugaresInteres['Direccion'][0]],[lugaresInteres['Latit'][0]],[lugaresInteres['Longit'][0]],[round(distancia,3)]]
datay=np.transpose(data)

# Creamos y rellenamos el dataframe de coordenadas para el Tableau con los datos del más cercano al piso
# ID : Codigo del lugar
# Nombre : nombre del lugar
# Direccion
# Latit y Longit : coordenas geográficas
# Distancia : distancia desde el piso seleccionado
dataID = pd.DataFrame(datay, columns=('ID','Nombre','Direccion','Latit','Longit','Distancia'))

# Para el resto de los lugares de interés :
for j in range(1,len(lugaresInteres)):
    distancia=round(abs(distGeo(latPiso,lonPiso,lugaresInteres['Latit'][j],lugaresInteres['Longit'][j])),3)
  # Preparamos los datos de los lugares de interes
    IDMenor = lugaresInteres['ID'][j]
    NombreMenor = lugaresInteres['Lugar'][j]
    DireccionMenor = lugaresInteres['Direccion'][j]
    LatitMenor = lugaresInteres['Latit'][j]
    LongitMenor = lugaresInteres['Longit'][j]   
    data=[IDMenor,NombreMenor,DireccionMenor,round(LatitMenor,6),round(LongitMenor,6),round(distancia,3)]
    datay=np.transpose(data)
    dataID.loc[j] = datay

In [0]:
# Por ultimo, añadimos los datos del mismo piso
airbnbMadridID = airbnbMadrid.loc[airbnbMadrid.loc[:, 'ID'] == IDPiso]
airLatitPiso=airbnbMadridID['Latitude'].values
airLongitPiso=airbnbMadridID['Longitude'].values

data=[IDPiso,'Piso','Ud.esta aqui',round(airLatitPiso[0],6),round(airLongitPiso[0],6),0]
datay=np.transpose(data)

dataID.loc[(len(lugaresInteres))] = datay

In [15]:
dataID

,ID,Nombre,Direccion,Latit,Longit,Distancia
0,LI01,Parque del Retiro,Parque del Retiro,40.415955,-3.684626,1.416
1,LI02,Palacio Real,Calle Bailen 1,40.417931,-3.713014,2.624
2,LI03,Gran Via,Gran via,40.420362,-3.705491,1.934
3,LI04,Puerta de Alcala,Plaza de la Independencia,40.419755,-3.68877,1.031
4,LI05,Mercado de San Miguel,Plaza de San Miguel,40.41543,-3.708758,2.464
5,LI06,Puerta del Sol,Puerta del Sol,40.416767,-3.703542,2.027
6,LI07,Museo del Prado,Calle Ruiz de Alarcón,40.41377,-3.69152,1.736
7,LI08,Plaza Mayor,Plaza Mayor,40.41548,-3.707487,2.376
8,LI09,Catedral de la Almudena,"Calle Bailén, 10",40.416318,-3.714745,2.838
9,LI10,Plaza de Cibeles,Plaza Cibeles,40.419275,-3.693031,1.228


Saca por fichero los datos reunidos

In [0]:
# Guarda en fichero de salida los datos recopilados.
dataID.to_excel('/content/drive/My Drive/1.Practica-Final/1.- Mensaje con sugerencias/airbnbMadridLugaresInteres.xlsx')